# A. Librerias y datos

In [8]:
import os
import time
import json
import requests
import pandas as pd

from bs4 import BeautifulSoup
from utils.paths import CHROMEDRIVER_PATH
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# B. Funciones

In [9]:
def set_google_maps_url(lat, lng):
    return f"https://www.google.com/maps/search/?api=1&query={lat},{lng}"

def set_street_view_url(lat, lng):
    return f"https://www.google.com/maps/search/?api=1&query={lat},{lng}"

In [10]:
def get_chrome_driver(chromedriver_path, print_view=False, headless=False):
    # Configurar las opciones de Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--ignore-certificate-errors")

    # Argumentos opcionales
    if print_view:
        options.add_argument("--disable-print-preview")
    if headless:
        options.add_argument("--headless=new")

    # Configurar el servicio del driver
    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service, options=options)
    
    return driver

# C. Prueba